In [0]:
class Bronze():
    def __init_(self, env):
        self. Conf = Config()
        self. landing zone = self. Conf.base_dir_data + "/raw"
        self. checkpoint_base = self.Conf.base_dir_checkpoint + "/checkpoints"
        self. catalog = env
        self.db_name = self. Conf. db_name
        spark. sql(f"USE {self. atalog}.{self :db_name}")
    def consume_user_registration (self, once=True, processing_time="5 seconds") :
        from pyspark.sql import functions as F
        schema = "user_id long, device_id long, mac_address string, registration_timestamp double"
        df_ stream = (spark. readStream
                    .format ("cloudFiles")
                    .schema (schema)
                    .option ("maxFilesPerTrigger", 1)
                    .option ("cloudFiles. format", "cs")
                    .option ("header", "true")
                    .load (self. landing_zone + "/registered_users_bz")
                    .withColumn ("load_time", F.current_timestamp ())
                    .withColumn ("source_file", F.input_file_name())
                    )
        stream writer = df_stream.writeStream \
                        .format ("delta") \
                        .option ("checkpointLocation", self. checkpoint_base + "/gym_ logins_bz") |
                        .outputMode ("append") \
                        .queryName ("gym_logins_bz_ingestion_ stream")
        spark. sparkContext.setLocalProperty ("spark.scheduler.pool", "bronze_p2")
        if once == True:
            return stream writer. trigger (availableNow=True). toTable(f" {self. catalog}.{self.db_name} - gym_logins_bz")
        else:
            return stream writer. trigger (processingTime=processing_time). toTable(f"{self.catalog}.{self.db_name}. gym_logins_bz")
        
    def consume_kafka_multiple(self, once-True, processing_time="5 seconds") :
        from pyspark.sql import functions as F
        schema = "key string, value string, topic string, partition bigint, offset bigint, timestamp bigint"
        df _date_lookup = spark. table(f" {self.catalog}. {self.db_name} date_lookup"). select ("date", "week_part")
        d_stream = (spark. readStream
                    .format ("cloudFiles")
                    .schema (schema)
                    .option ("maxFilesPerTrigger", 1)
                    .option ("cloudFiles. format", "json")
                    .load (self. landing_zone + "/kafka_multiplex_bz")
                    .withColumn ("load_time", F.current_timestamp ())
                    .withColumn ("source_file", F. input_file_name ())
                    .join (F. broadcast (df _date_lookup),
                        [F. to _date((F.col ("timestamp") /1000). cast ("timestamp")) == F. col ("date")],
                    "left")
        # Use append mode because bronze layer is expected to insert only from source
        stream writer = df_stream.writeStream/
                        .format("delta") \
                        .option ("checkpointLocation", self.checkpoint_base + "/kafka_multiplex_bz") \
                        .outputMode ("append" ) \
                        .queryName ("kafka_multiplex_bz_ingestion_stream")